In [1]:
import numpy as np
from skimage import io
from matplotlib import pyplot as plt
#from deepcell.datasets import multiplex_tissue
#from deepcell.utils.plot_utils import create_rgb_image
#from PIL import Image
import pandas as pd
import tifffile
import imageio
import glob
import multiprocessing as mp
import re

/mnt/isilon/tan_lab/bandyopads/miniconda3/envs/Mesmer_0.10/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
mcmicroDir = '/mnt/isilon/tan_lab_imaging/FUSION/NBM27_H10_CITRATE_REIMAGE/H10/Scan1'
inOmeTiff = glob.glob(mcmicroDir + '/*qptiff')[0]
# Make sure to adjust this to nuclear or whole cell depending on what's needed
inMaskTiff = glob.glob(mcmicroDir + '/mesmer/*_segmentation_cell.tif')[0]

In [3]:
def sumValueByMask(mask, expr, outFile = ""):
    df = pd.DataFrame({
        'values':  np.concatenate(expr),
        'indexes': np.concatenate(mask)
    })
    out = df.pivot_table(values='values', index='indexes', aggfunc=sum)
    if(outFile != ""):
        out.to_csv(outFile)
    return out

In [4]:
im = tifffile.imread(inOmeTiff)
im.shape

(54, 24480, 16320)

In [5]:
mask = tifffile.imread(inMaskTiff)

df = pd.DataFrame({
  'values': np.concatenate(np.full(mask.shape,1)),
  'indexes': np.concatenate(mask)
})

size = sumValueByMask(mask, np.full(mask.shape,1), mcmicroDir + '/mesmer/cell_size.csv')

coordX = sumValueByMask(
    mask, 
    np.tile(
        np.array(range(0,mask.shape[1])), 
        (mask.shape[0], 1)),
    mcmicroDir + '/mesmer/cell_x_coord.csv')

coordY = sumValueByMask(
    mask, 
    np.transpose(np.tile(
        np.array(range(0,mask.shape[0])), 
        (mask.shape[1], 1))),
    mcmicroDir + '/mesmer/cell_y_coord.csv')   

outFiles = [mcmicroDir + '/mesmer/channel_C{:02d}.csv'.format(ii) for ii in range(im.shape[0])]

In [6]:
for ii in range(len(outFiles)):
    print(ii)
    sumValueByMask(mask, im[ii,:,:], outFiles[ii])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53


In [7]:
# Create a super-csv from segmentation results for downstream analysis
import os
csvDir = os.path.join(mcmicroDir + '/mesmer')
csvFileArray = sorted(glob.glob(csvDir + "/*.csv"))
print("Output Directory is " + csvDir)
df_list = []
for filename in csvFileArray:
    df = pd.read_csv(filename, header=0)
    df = df.rename(columns={"values": str(filename)}) # changes "values" to the filename
    df_list.append(df)
col_names = [str(i) for i in csvFileArray]
dfs = [df.set_index('indexes') for df in df_list]
frame = pd.concat(dfs, axis=1)
print(np.shape(frame)) # to confirm that the data-frame has written the correct number of columns
frame.to_csv(os.path.join(csvDir + 'combined_markers.csv'), index=True)

Output Directory is /mnt/isilon/tan_lab_imaging/FUSION/NBM27_H10_CITRATE_REIMAGE/H10/Scan1/mesmer
(264067, 57)


In [8]:
# For checking the segmentation quality
from deepcell.utils.plot_utils import create_rgb_image
tif_for_mesmer = glob.glob(mcmicroDir + '/mesmer/*_nuclear_panmembrane.ome.tif')[0]
im = io.imread(tif_for_mesmer)
X_train = np.transpose(im, (1,2,0))
X_train = X_train.reshape(1, X_train.shape[0], X_train.shape[1], X_train.shape[2])
rgb_images = create_rgb_image(X_train, channel_colors=['green', 'blue'])

segmentation_predictions = np.load("/mnt/isilon/tan_lab_imaging/FUSION/NBM26/mesmer/segmentation_predictions.npy")
# segmentation_predictions_nuc = np.load("/mnt/isilon/tan_lab_imaging/Analysis/bandyopads/SB27_AML06_UpdatedPanel_Test/regions/reg001/mesmer/segmentation_predictions_nuclear.npy")

from deepcell.utils.plot_utils import make_outline_overlay
idx = 0
overlay_data = make_outline_overlay(rgb_data=rgb_images, predictions=segmentation_predictions)
fig, ax = plt.subplots(1, 2, figsize=(15, 15))
ax[0].imshow(rgb_images[idx, 2000:2500, 1000:1500,])
ax[1].imshow(overlay_data[idx,2000:2500, 1000:1500,])
ax[2].imshow(rgb_images['blue'])
ax[0].set_title('Raw data')
ax[1].set_title('Predictions')
plt.show()
borders = overlay_data[0, ...]-rgb_images[idx, ...]


ImportError: cannot import name 'register_translation' from 'skimage.feature' (/mnt/isilon/tan_lab/bandyopads/miniconda3/envs/Mesmer_0.10/lib/python3.8/site-packages/skimage/feature/__init__.py)